In [1]:
#######################
# Project 3
# ESE 572
# Layer 2 Channel Coding
#######################

# Choose a probability of bit error to use when simulating the receiving end.

# prob = 0.1
prob = 0.01

In [2]:
#######################
# Step 1
#######################

# Function to convert string to binary using ASCII encoding
def string_to_binary(string):
    binary = ''
    binaryS = ''
    binaryC = ''
    counter = 0
    for char in string:
        ascii_code = ord(char) # Get ASCII code of character
        if counter == 24:
            binary += format(ascii_code, '08b')
            binaryS += format(ascii_code, '08b') + ' '
            binaryC += format(ascii_code, '08b') + '\n'
            counter = 0
        else:
            binary += format(ascii_code, '08b')
            binaryS += format(ascii_code, '08b') + ' '
            binaryC += format(ascii_code, '08b')
            counter += 1
    return binary, binaryS, binaryC

# Load the document
filename = 'input.txt'
text = ''
with open(filename,'r') as i:
    text += i.readline()
    
binary_data, binaryS_data, binaryC_data = string_to_binary(text)

# binary_data is single line
# binaryS_data contain spaces to delimitate each character
# binaryC_data is split as chunks of 200 bits


In [3]:
#######################
# Step 2
#######################

# Create CRC for g(D) = [D16 + D12 + D5 + 1]
crc_str = '10001000000100001' # divisor
int_crc_str = int(crc_str,2)
frames = []

binary_chunks = binaryC_data.split('\n')
for chunk in binary_chunks[:-1]:
    temp = chunk + "0000000000000000"
    temp = int(temp,2)
    crc = temp % int_crc_str
    crc = format(crc, '016b')
    if len(crc) > 16:
        crc = crc[-16:]
    frames.append(chunk + crc)

print(len(frames[0]))

216


In [4]:
#######################
# Step 3
#######################
## Channel Coding **NEW**

import numpy as np

def encode_message(msg, matrix):
    code = np.mod(np.dot(msg, matrix), 2)
    return code

def decode_message(code, valid_codewords):
    incorrect_vec = []
    incorrect = 0
    for codeword in valid_codewords:
        for ind in range(len(codeword)):
            if codeword[ind] != code[ind]:
                incorrect += 1
        incorrect_vec.append(incorrect)
        incorrect = 0
    closest = np.min(incorrect_vec)
    closest = np.where(incorrect_vec == closest)[0][0]
    decoded_msg = valid_data[closest]
    return decoded_msg

G = np.array([[1,1,1,1,1,1,1,1],
              [0,0,0,0,1,1,1,1],
              [0,0,1,1,0,0,1,1],
              [0,1,0,1,0,1,0,1]])

valid_data = [[0,0,0,0], [0,0,0,1], [0,0,1,0], [0,0,1,1],
              [0,1,0,0], [0,1,0,1], [0,1,1,0], [0,1,1,1],
              [1,0,0,0], [1,0,0,1], [1,0,1,0], [1,0,1,1],
              [1,1,0,0], [1,1,0,1], [1,1,1,0], [1,1,1,1]]

valid_codewords = []
for data in valid_data:
    valid_codewords.append(encode_message(data,G))
print(valid_codewords)

# Example
msg = np.array([0, 1, 1, 1])
print("Sent Message:", msg)
print()

code = encode_message(msg, G)
print("Encoded code:", code)
print()

decoded_msg = decode_message(code, valid_codewords)
print("Decoded message:", decoded_msg)
# End Example

def array_to_string(array):
    output = ""
    for each in array:
        output += str(each)
    return output

def string_to_array(string):
    output = []
    for each in string:
        output.append(int(each))
    output = np.array(output)
    return output

def RM_string(input_string):
    output_string = ""
    ind = 0
    while(ind <= len(input_string)):
        dat = input_string[ind:ind+4]
        data_string = string_to_array(dat)
        if data_string.size != 0:
            data_string = data_string.reshape((1,4))
            encoded = encode_message(data_string,G)[0]
            output_string += array_to_string(encoded)
        else:
            output_string += str(dat[-1:])
        ind += 4
    return output_string

binaryRM = []
for frame in frames:
    binaryRM.append(RM_string(frame))
    
print(len(binaryRM[0]))

[array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 1, 0, 1, 0, 1, 0, 1]), array([0, 0, 1, 1, 0, 0, 1, 1]), array([0, 1, 1, 0, 0, 1, 1, 0]), array([0, 0, 0, 0, 1, 1, 1, 1]), array([0, 1, 0, 1, 1, 0, 1, 0]), array([0, 0, 1, 1, 1, 1, 0, 0]), array([0, 1, 1, 0, 1, 0, 0, 1]), array([1, 1, 1, 1, 1, 1, 1, 1]), array([1, 0, 1, 0, 1, 0, 1, 0]), array([1, 1, 0, 0, 1, 1, 0, 0]), array([1, 0, 0, 1, 1, 0, 0, 1]), array([1, 1, 1, 1, 0, 0, 0, 0]), array([1, 0, 1, 0, 0, 1, 0, 1]), array([1, 1, 0, 0, 0, 0, 1, 1]), array([1, 0, 0, 1, 0, 1, 1, 0])]
Sent Message: [0 1 1 1]

Encoded code: [0 1 1 0 1 0 0 1]

Decoded message: [0, 1, 1, 1]
432


In [5]:
#######################
# Step 4
#######################

# flag = '01111110'
flag = '000111111111111111111000' # tripled
framed_frames = []
bitstuff=0
for binRM in binaryRM:
    i = 0
    
    while(i <= (len(binRM) - 18)):
        if binRM[i:i+17] == '11111111111111111':
            binRM = binRM[:i+16] + '0' + binRM[i+16:] # bit stuff after any string of five 1s
            bitstuff+=1
        i += 8
    framed_frames.append(flag + binRM)
    
framed_frames.append(flag) # after last FEC frame, insert another flag
print('bits stuffed: '+str(bitstuff))
print(framed_frames[0])

bits stuffed: 0
000111111111111111111000000011111010101000111100110000110011001100000000011001100101010101100110111111110110011001011010011001100110011000110011111100000011001100000000011010010000000001101001001100110011110010010110001111001010010100111100101010100011110011000011001111000101101000111100110000110110100100001111001100110000000001011010011001100110100100001111001100111100001100110011000000000000111111110000001111001001011010010110111111111001100111001100


In [6]:
#######################
# Sequence to Transmit
#######################

sequence = ''
for fram in framed_frames:
    sequence += fram
    # print(len(fram))
    
print(len(sequence))

20544


In [7]:
#######################
# Step 5: Error
#######################

import random

lim = {0.1: 9,
       0.01: 99}

rxbits = ''
un = 0

for bit in sequence:
    # rnum = random.randint(0,lim[prob])
    rnum = 1 # no errors
    if rnum == 0: # error bit
        un += 1
        if bit == '1':
            bit = '0'
        else:
            bit = '1'
    rxbits += bit
print(un)
print(len(rxbits))

0
20544


In [8]:
#######################
# Step 5: Unstuffing
#######################

# stuff_seq = '000111110'

def unstuff(bits):
    i = 0
    while i < len(bits)-3:
        #print(bits[i:i+19])
        if bits[i:i+18] == '111111111111111101':
            print(bits[i:i+18])
            bits = bits[:i+16] + bits[i+17:] # remove bit stuffing
            print(bits[i:i+18])
            i += 19
        else:
            i += 8
    return bits

rxbits = unstuff(rxbits)
# rxbits, un = unstuff(sequence)
print(len(rxbits))

20544


In [9]:
#######################
# Step 5: Fixing
#######################

corrected_bits = rxbits
# finding flags to reseparate frames
i = 0
start = -1
end = -1
new_frames = []
flag_flag = 0

while (i < len(corrected_bits)):
    if corrected_bits[i:i+24] == flag:
        flag_flag = 1
        if start != -1:
            end = i
            temp = corrected_bits[start:end]
            new_frames.append(temp)
            start = i
        else:
            # temp = corrected_bits[:i]
            # new_frames.append(temp)
            start = i
    if flag_flag:
        i += 24
        flag_flag = 0
    else:
        i += 1
new_frames.append(corrected_bits[start:])
# new_frames = new_frames[1:]

print(new_frames[-1])
print(len(new_frames[-2]))
# frames separated, unstuffed but need :: untripled, CRC checked, and then converted to text

000111111111111111111000
456


In [10]:
#######################
# Step 5: Remove flag
#######################

rxseqs = []
for nf in new_frames:
    rxseqs.append(nf[24:])

rxseqs = rxseqs[0:-1]
print(len(rxseqs[0]))
# need :: CRC checked, and then converted to text

432


In [11]:
#######################
# Step 5: Decode RM
#######################

temp_rxseqs = []
for frame in rxseqs:
    temp_frame = ""
    count = 0
    while(count+8 < len(frame)):
        message = string_to_array(frame[count:count+8])
        decoded_msg = decode_message(message, valid_codewords)
        temp_frame += array_to_string(decoded_msg)
        count += 8
    temp_frame += frame[count:]
    temp_rxseqs.append(temp_frame)
    print(temp_frame)

rxseqs = temp_rxseqs
print(rxseqs)

...']


In [13]:
#######################
# Step 5: Check CRC
#######################

# Create CRC for g(D) = [D16 + D12 + D5 + 1]
crc_str = '10001000000100001' # divisor
int_crc_str = int(crc_str,2)
failed = 0
total = 0

for rxs in rxseqs:
    crc_rx = rxs[-16:]
    beginning = rxs[:-16] + "0000000000000000"
    beginning = int(beginning,2)
    crc = beginning % int_crc_str
    crc = format(crc,'016b')
    if len(crc) > 16:
        crc = crc[-16:]
    if crc != crc_rx:
        print("crc: "+str(crc)+" crc_rx: "+str(crc_rx))
        failed += 1
    total += 1

print('Number of Failed Frames with Error Probability ' + str(prob) + ': --- ' + str(failed) + ' --- \n')
print('Failed Frames: ' + str((failed / total) * 100) + '%' + ' of ' + str(total) + ' total frames recieved.')

crc: 1100100000000100 crc_rx: 1000101111001100
crc: 1001101111001100 crc_rx: 1101110001011010
crc: 0100110010110110 crc_rx: 1110111000000000
crc: 0010010001011111 crc_rx: 0011011010101010
crc: 0110011010000100 crc_rx: 0010101000110011
crc: 1101000110011110 crc_rx: 1111000111000011
crc: 0100011101011011 crc_rx: 1001101001010101
crc: 1100100011100101 crc_rx: 0111101001011010
crc: 0000111010110000 crc_rx: 0000111010011001
crc: 1000101011100011 crc_rx: 0101110010101010
crc: 1110000001110000 crc_rx: 1110000001101001
crc: 1000101000011110 crc_rx: 1010101000111100
crc: 1100011001001110 crc_rx: 1110011010101010
crc: 0011110010110000 crc_rx: 0011110010011001
crc: 0111010010101111 crc_rx: 1000011011000011
crc: 0011001000110111 crc_rx: 1100010101101001
crc: 1111110110100100 crc_rx: 1100000100001111
crc: 1010101101000000 crc_rx: 1010101100001111
crc: 0011010001000000 crc_rx: 0011010000001111
crc: 1101111111011000 crc_rx: 0110001010010110
crc: 0000011100100101 crc_rx: 1011100010101010
crc: 11100011

In [14]:
#######################
# Convert Back to Text
#######################

all_rx_bits = ''
char_arr = []
paragraph = ''
i = 0

#print(rxseqs)

for rxs in rxseqs:
    print(lens(rxs[:-15]))
    if len(rxs[:-15]) < 180:
        continue
    else:
        print("HERE")
        all_rx_bits += rxs[:-15]
    
while i < len(all_rx_bits)-7:
    char_arr.append(all_rx_bits[i:i+8])
    i += 8

for cb in char_arr:
    character = chr(int(cb,2))
    if (character!='~'):
        paragraph += character

print(paragraph)

In 1853, prominent St. Loû«Kk+C
s¡»
Ëk
q{¹X[\È
\ÝÜÚ[X[HÖäÊÊÜØÊÂÌ@ØÒÞè@ä\X@ÆÞÜÆË'&æVB&÷WBFRÆ6²öbï9º4º:º4··97³44³´2¹62°¹
¹¥¹¥¸Ñ¡É½Ý¥¹µ¥Ýîn

¬
¤
Íî­Ì-Ìä
ìÄ
ìashington University in S¡qc{«Kq#«Ks9£C)Á¡È[
L
Ë
Ø]\ÈÙ[[ZYÂäÂÜèæ@ÌØÞÞÈÊÈ@ÒÜèÞ@¦è\@Þ7V2Â&ö÷7FærFR÷VÆA´··7³:42<·º·31´º<+´Ñ Ñ¡Í¹Ý½µÉÌ
µdL®nm-Ìä
Ì¬¬
ÍîD
¬¬l.ion - both industrial traûKsKs9
s!
K;+s+
a[Ý\Ù\È
HÛÛ
XÝYÝ]
ÚY Ê@ÞÌ@ÜÞäÚÂØ@îÞäÖÒÜÎ@ÐÞêäç¢â6òFRf'7BVGV6Föæ¶9º2¸7³:42<·º·3+°¹´4á¹Ñ½¸U¹¥ÙÉÍ¥ÑäÝ
ÌÑ¼n,M.m
-Ä
®Ì­Í-ÌäMìîDam on October 22, 1854. OË³+£C)«++#Ks9#+
#)Ë
HÛÛ[Z[ÈY
XØ]bÞÜ@àäÞÎäÂÚ@êÜÈÊäîÊÜè@ÚÂÜòâ6ævW2âFRVæfW'6G367º¹4¹´2²0º4º967±°º4·¹¸¥¸½Ý¹Ñ½Ý¸MÐ¸1½Õ¥ÌíîD
.d
Í.Nn¦,¬.Ne
îLowing from an evening pro;
i£y
qKs£K£«£K{q{0\[ÈH[
Û]HÙ
ØúÒÊÜèÒÌÒÆX@ØÒÄÊäÂØ@Âäèæ@ÂÜB6Æ766Â6÷W'6RöffW&73¹$·:4¶²9±´7·¶9

In [15]:
# In 1853, prominent St. Louis merchant Wayman Crow and his pastor, William Greenleaf Eliot Jr., concerned about 
# the lack of institutions of higher learning in the growing midwest, led the founding of Washington University  
# in St. Louis. During the 1840s and 50s, waves of immigrants flooded into St. Louis, boosting the population of 
# the young city. With these newcomers came a pressing need for education - both industrial training and basic 
# general courses - conducted outside of normal working hours. So the first educational step of the young 
# Washington University was to establish an evening program on October 22, 1854. Over the succeeding decades, 
# the continuing education program underwent many changes. The university flourished at its location in downtown 
# St. Louis for its first 50 years, growing from an evening program to an institution offering a full slate of 
# scientific, liberal arts and classical course offerings. In time, schools of law and fine arts were added. In 
# 1891, the school acquired the St. Louis Medical College to form a medical department, which merged with the 
# Missouri Medical College in 1899.